<a href="https://colab.research.google.com/github/gowun/BladderCancer_AMC/blob/2-gowun/Notebooks/Data_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.colab import drive
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')
home_path = '/content/gdrive/My Drive/BladderCancer_AMC/'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone -b 2-gowun https://github.com/gowun/BladderCancer_AMC.git

Cloning into 'BladderCancer_AMC'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 79 (delta 27), reused 54 (delta 17), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [3]:
from BladderCancer_AMC.ModelingTools import utils as ut
from BladderCancer_AMC.ModelingTools import preprocessing as pp
from BladderCancer_AMC.ModelingTools import figure as fg

In [4]:
ds_names = ['mda_mvac', 'mda_ddmvac', 'meta_datasets', 'amc']
datasets4 = list(map(lambda x: ut.load_data(f'{home_path}{x}.pkl', 'pickle'), ds_names))

1. 모든 데이터셋에 존재하는 컬럼 (intersect_cols) 및 각 데이터 내 intersect_cols 비율 

In [5]:
#### 전체 데이터셋 모두에서 등장하는 컬럼셋
datasets4_cols = list(map(lambda x: set(x.columns), datasets4))
intersect_cols = datasets4_cols[0]
for cols in datasets4_cols[1:]:
  intersect_cols = sorted(set(intersect_cols).intersection(cols))
len(intersect_cols)

17000

In [15]:
intersect_cols.index('response')

16999

In [7]:
## 데이터셋 별 intersect_cols 의 비율
for i, cols in enumerate(datasets4_cols):
  r = round(len(intersect_cols) / len(cols) * 100.0, 2)
  print(f'{ds_names[i]}: {r}% from {len(cols)} Gene Symbols')

mda_mvac: 81.98% from 20737 Gene Symbols
mda_ddmvac: 81.7% from 20808 Gene Symbols
meta_datasets: 36.94% from 46023 Gene Symbols
amc: 69.49% from 24463 Gene Symbols


2. 주어진 각 Classifiers 중 intersect_cols에 포함된 비율

In [8]:
## Classifier.csv 로드 및 정제
clfs = ut.load_data(home_path + 'Classifiers.csv', 'csv').T

In [10]:
clfs_dict = dict()
for i, row in enumerate(clfs.iloc[:-1].values):
  tmp = row[np.array(clfs.iloc[i].isna() == False)]
  tmp1 = list(filter(lambda x: x in intersect_cols, tmp))
  k_name = list(clfs.index)[i]
  clfs_dict[k_name] = [len(tmp1), len(tmp1) / len(tmp), tmp1]
  #if len(tmp1) < len(tmp):
  print(f'{k_name}: {clfs_dict[k_name][:2]}')

Metacore&GSEA target genes: [26, 0.8387096774193549]
GSEA_R enrichment_all: [37, 0.9736842105263158]
GSEA_R enrichment_intersection: [11, 1.0]
BC Basal: [19, 1.0]
BC basal-9gene: [9, 1.0]
BC Luminal: [19, 1.0]
BC Luminal-16gene: [16, 1.0]
BC P53 pathway: [95, 0.9895833333333334]
BC P53like: [19, 1.0]
BC P63 pathway: [39, 1.0]
Basal-squamous_ref.eau-cancer genome: [93, 0.96875]
Luminal_ref.eau-cancer genome: [42, 0.9545454545454546]
Luminal-infiltrated_ref.eau-cancer genome: [71, 1.0]
Luminal-papillary_ref.eau-cancer genome: [23, 1.0]
Neuronal_ref.eau-cancer genome: [94, 0.9690721649484536]


In [11]:
ut.save_data(clfs_dict, home_path + 'intersect_classifiers.pkl', 'pickle')

In [12]:
clfs_df = pd.DataFrame(list(map(lambda x: x[2], clfs_dict.values())), index=list(clfs_dict.keys())).T
ut.save_data(clfs_df, home_path + 'intersect_classifiers.csv', 'csv')

3. 각 데이터셋의 기초 통계

In [13]:
%%time
mean_datasets4 = list(map(lambda x: pp.handle_repeated_columns(x[intersect_cols])[1], datasets4))

CPU times: user 48.5 s, sys: 1.08 s, total: 49.6 s
Wall time: 47.9 s


In [14]:
ut.save_data(mean_datasets4, f'{home_path}mean_datasets4.pkl', 'pickle')

In [16]:
%%time
for i, nm in enumerate(ds_names):
  tmp = mean_datasets4[i].describe()
  ut.save_data(tmp.reset_index(), f'{home_path}{nm}.csv', 'csv')

CPU times: user 2min 12s, sys: 499 ms, total: 2min 13s
Wall time: 2min 15s


4. Optimal Normalization Algorithm 도출

In [17]:
methods = ['log2', 'log2_minmax', 'standard', 'minmax', 'max_abs', 'robust', 'power', 'quantile', 'rankgauss']

In [18]:
%%time
scaled_datasets4 = []
for i, d in enumerate(mean_datasets4):
  print(i)
  scaled_datasets4.append(pp.scale_with_scalers(d, methods))

0


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log2

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning:

overflow encountered in multiply

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:199: RuntimeWarning:

overflow encountered in reduce

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2947: RuntimeWarning:

overflow encountered in power

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning:

n_quantiles (1000) is greater than the total number of samples (23). n_quantiles is set to n_samples.



1


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log2

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2947: RuntimeWarning:

overflow encountered in power

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning:

overflow encountered in multiply

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:199: RuntimeWarning:

overflow encountered in reduce

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning:

n_quantiles (1000) is greater than the total number of samples (38). n_quantiles is set to n_samples.



2


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

invalid value encountered in log2

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log2

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:375: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:376: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning:

n_quantiles (1000) is greater than the total number of samples (299). n_quantiles is set to n_samples.



3


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log2

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning:

n_quantiles (1000) is greater than the total number of samples (43). n_quantiles is set to n_samples.



CPU times: user 6min 43s, sys: 7.15 s, total: 6min 50s
Wall time: 6min 36s


In [19]:
from itertools import combinations
orders = list(range(len(mean_datasets4)))
orders = list(combinations(orders, 2))
orders

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [20]:
%%time
## 알고리즘 별로 분포가 비슷한 컬럼셋 도출
cols_dict = dict()
remain_clfs_dict = dict()
for m in methods:
  print(m)
  for o1, o2 in orders:
    if m in cols_dict.keys():
      cols = sorted(set(intersect_cols[:-1]).intersection(set(cols_dict[m])))
      if len(cols) > 0:
        cols_dict[m] = pp.compare_two_samples_and_draw_feasible_columns(scaled_datasets4[o1][m][cols], scaled_datasets4[o2][m][cols])
    else:
      cols_dict[m] = pp.compare_two_samples_and_draw_feasible_columns(scaled_datasets4[o1][m][:-1], scaled_datasets4[o2][m][:-1])
  print(f'{m}: {len(cols_dict[m])} remains')
  if len(cols_dict[m]) > 0:
    remain_clfs_dict[m] = dict()
    for key in clfs_dict.keys():
      rrr = sorted(set(clfs_dict[key][2]).intersection(set(cols_dict[m])))
      rr = len(rrr) / clfs_dict[key][0] * 100.0
      r = clfs_dict[key][1] * rr
      remain_clfs_dict[m][key] = [rrr, rr, r]
      print(f'{m}-{key}: {len(rrr)}({r} % from the original, {rr} % from the reduced) remaines')

log2
log2: 0 remains
log2_minmax
log2_minmax: 13 remains
log2_minmax-Metacore&GSEA target genes: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-GSEA_R enrichment_all: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-GSEA_R enrichment_intersection: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC Basal: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC basal-9gene: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC Luminal: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC Luminal-16gene: 1(6.25 % from the original, 6.25 % from the reduced) remaines
log2_minmax-BC P53 pathway: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC P53like: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-BC P63 pathway: 0(0.0 % from the original, 0.0 % from the reduced) remaines
log2_minmax-Basal-squamous_ref.eau-ca

In [21]:
%%time
remain_mth_datasets = dict()
for m in remain_clfs_dict.keys():
  if sum(map(lambda x: x[1] == 100.0, remain_clfs_dict[m].values())) == len(clfs_dict):
    print(m)
    tmp = []
    for ll in scaled_datasets4:
      tmp.append(ll[m][cols_dict[m] + ['response']])
    remain_mth_datasets[m] = tmp

standard
power
rankgauss
CPU times: user 183 ms, sys: 37.7 ms, total: 221 ms
Wall time: 218 ms


In [22]:
ut.save_data(remain_clfs_dict, f'{home_path}scaled_datasets_3mths.pkl', 'pickle')